Setting GPU

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
from tqdm import tqdm
import cv2
import os
import seaborn as sns
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True) 

Section 1: Run all code blocks in this section to build and train a Vanilla GAN model with pituitary dataset

In [ ]:
# load and preprocess the dataset

dataset3 = tf.keras.preprocessing.image_dataset_from_directory(directory='C:/brain_tumor/version2/classify_dataset/meningioma', label_mode=None, batch_size=8, image_size=(256, 256), shuffle=True, color_mode='grayscale')
dataset3 = dataset3.map(lambda x: (x - 127.5) / 127.5)
dataset3 = dataset3.cache()
dataset3 = dataset3.prefetch(16)

In [ ]:
# build the generator

from tensorflow import keras
from keras.layers import BatchNormalization, Conv2D, Dense, Flatten, Reshape, LeakyReLU, Dropout, UpSampling2D, Conv2DTranspose
from keras.models import Sequential

def dc_generator():

    model = Sequential()

    model.add(Dense(16*16*256, input_dim=256))
    # model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((16, 16, 256)))
    
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', use_bias=False))
    # model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2DTranspose(64, (4,4), strides=(2,2), padding='same', use_bias=False))
    # model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2DTranspose(32, (4,4), strides=(2,2), padding='same', use_bias=False))
    # model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    
    model.add(Conv2DTranspose(1, (4,4), strides=(2,2), padding='same',use_bias=False, activation='tanh'))
    
    return model

dc_generator = dc_generator()
dc_generator.summary()

In [ ]:
# generate some random noises to test the generator

image_dc = dc_generator.predict(np.random.randn(4, 256))
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(image_dc):
    for i in range(4):
        ax[idx].imshow(np.squeeze(image_dc[i]), cmap='gray')
        ax[idx].title.set_text(idx)

In [ ]:
# build the discriminator

def dc_discriminator(input=(256,256,1)):

    model = Sequential()
    
    model.add(Conv2D(32, (4,4), strides=(2, 2), padding='same',input_shape=input))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    
    model.add(Conv2D(64, (4,4), strides=(2, 2), padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    
    model.add(Conv2D(128, (4,4), strides=(2, 2), padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    
    model.add(Conv2D(256, (4,4), strides=(2, 2), padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    return model

dc_discriminator = dc_discriminator()
dc_discriminator.summary()

In [ ]:
# test the discriminator

dc_discriminator.predict(image_dc)

In [ ]:
# setting the optimizers and losses

from tensorflow import keras
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy
from keras.models import Model
from keras.callbacks import Callback
import os

dc_d_optimizer=Adam(lr=0.00003)
dc_g_optimizer=Adam(lr=0.0002)
d_loss=BinaryCrossentropy()
g_loss=BinaryCrossentropy()

In [ ]:
# training process 

class DCGAN(Model):
    def __init__(self, dc_generator, dc_discriminator):
        super().__init__()
        self.dc_generator = dc_generator
        self.dc_discriminator = dc_discriminator
    #     self.g_loss_metric = keras.metrics.Mean(name='g_loss')
    #     self.d_loss_metric = keras.metrics.Mean(name='d_loss')
    
    def compile(self, dc_g_optimizer, dc_d_optimizer, d_loss, g_loss):
        super(DCGAN, self).compile()
        self.dc_g_optimizer = dc_g_optimizer
        self.dc_d_optimizer = dc_d_optimizer
        self.d_loss = d_loss
        self.g_loss = g_loss
        # self.loss_fn = loss_fn
        
    def train_step(self, real_images):
        # get batch size from the data
        batch_size = tf.shape(real_images)[0]
        # generate random noise
        random_noise = tf.random.normal(shape=(batch_size, 256))
        
        # train the discriminator with real (1) and fake (0) images
        with tf.GradientTape() as tape:
            # compute loss on real images
            pred_real = self.dc_discriminator(real_images, training=True)
            # generate real image labels
            real_labels = tf.ones((batch_size, 1))
            # label smoothing
            real_labels -= 0.15 * tf.random.uniform(tf.shape(real_labels))
            d_loss_real = self.d_loss(real_labels, pred_real)
            
            # compute loss on fake images
            fake_images = self.dc_generator(random_noise)
            pred_fake = self.dc_discriminator(fake_images, training=True)
            # generate fake labels
            fake_labels = tf.zeros((batch_size, 1))
            fake_labels += 0.15 * tf.random.uniform(tf.shape(fake_labels))
            d_loss_fake = self.d_loss(fake_labels, pred_fake)
            
            # total discriminator loss
            dc_d_loss = (d_loss_real + d_loss_fake) / 2
            
        # compute discriminator gradients
        gradients = tape.gradient(dc_d_loss, self.dc_discriminator.trainable_variables)
        # update the gradients
        self.dc_d_optimizer.apply_gradients(zip(gradients, self.dc_discriminator.trainable_variables))
        
        # train the generator model
        labels = tf.ones((batch_size, 1))
        with tf.GradientTape() as tape:
            # generate fake images from generator
            fake_images = self.dc_generator(random_noise, training=True)
            # classify images as real or fake
            pred_fake = self.dc_discriminator(fake_images, training=True)
            # compute loss
            dc_g_loss = self.g_loss(labels, pred_fake)
            
        # compute gradients
        gradients = tape.gradient(dc_g_loss, self.dc_generator.trainable_variables)
        # update the gradients
        self.dc_g_optimizer.apply_gradients(zip(gradients, self.dc_generator.trainable_variables))
        
        # # update states for both models
        # self.d_loss_metric.update_state(dc_d_loss)
        # self.g_loss_metric.update_state(dc_g_loss)
        
        return {'d_loss': dc_d_loss, 'g_loss': dc_g_loss}

In [ ]:
# create a callback after each epoch

class DCGANMonitor(Callback):
    def __init__(self, num_imgs=3, latent_dim=256):
        self.num_imgs = num_imgs
        self.latent_dim = latent_dim
        # create random noise for generating images
        # self.noise = tf.random.normal([4, latent_dim])

    def on_epoch_end(self, epoch, logs=None):
        # generate the image from noise
        noise = tf.random.normal([self.num_imgs, self.latent_dim])
        g_img = self.model.dc_generator(noise)
        # denormalize the image
        g_img = (g_img * 127.5) + 127.5
        g_img.numpy()
        
        fig = plt.figure(figsize=(20, 20))
        for i in range(self.num_imgs):
            plt.subplot(5, 5, i+1)
            img = tf.keras.utils.array_to_img(g_img[i])
            # img.save(os.path.join('output_dcgan2', f'generated_img_from150_{epoch}_{i}.png'))
            plt.imshow(img, cmap='gray')
            plt.axis('off')
        # plt.savefig('epoch_{:03d}.png'.format(epoch))
        plt.show()

In [ ]:
# training model

brain_tumor_dc = DCGAN(dc_generator, dc_discriminator)
brain_tumor_dc.compile(dc_g_optimizer, dc_d_optimizer, d_loss, g_loss)
N_EPOCHS = 200
brain_tumor_dc.fit(dataset3, epochs=N_EPOCHS, callbacks=[DCGANMonitor()])

Section 2: Using trained model to generate images

In [ ]:
# call trained model

from keras.models import load_model
load_dc_generator12 = load_model('meni_model/gen12(2000e).h5')
load_dc_discriminator12 = load_model('meni_model/dis12(2000e).h5')
dc_model12 = DCGAN(load_dc_generator12, load_dc_discriminator12)
dc_model12.compile(dc_g_optimizer, dc_d_optimizer, d_loss, g_loss)

In [ ]:
# generate images

import os
import tensorflow as tf
import matplotlib.pyplot as plt

noise = tf.random.normal([50, 256])
g_img = dc_model12.dc_generator(noise)
g_img = (g_img * 127.5) + 127.5
g_img_numpy = g_img.numpy()
save_dir = 'C:/brain_tumor/version2/notebook/dc_gan_final/meni/meni_img'
os.makedirs(save_dir, exist_ok=True)
# fig, ax = plt.subplots(ncols=4, nrows=4, figsize=(20, 20))
for i in range(50):
    # ax[i // 4][i % 4].imshow(g_img_numpy[i, :, :, 0], cmap='gray') 
    # ax[i // 4][i % 4].axis('off')
    file_path = os.path.join(save_dir, f'images_6_{i}_1.png')
    tf.keras.preprocessing.image.save_img(file_path, g_img_numpy[i])
# plt.tight_layout()
# plt.show()

Section 3: Evaluate generated images by metrics

In [ ]:
# SSIM metric

import os
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim

real_images_folder = 'C:/brain_tumor/version2/notebook/classify_dataset/meningioma'
generated_images_folder = 'C:/brain_tumor/version2/notebook/dc_gan_final/meni/meni_img'

def load_images_from_folder(folder):
    images = []
    image_paths = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            images.append(img)
            image_paths.append(img_path)
    return images, image_paths

real_images, real_image_paths = load_images_from_folder(real_images_folder)

generated_images, generated_image_paths = load_images_from_folder(generated_images_folder)

highest_ssim_values = []

for gen_img, gen_img_path in zip(generated_images, generated_image_paths):
    max_ssim = -1
    best_real_img_path = None
    for real_img, real_img_path in zip(real_images, real_image_paths):
        if gen_img.shape != real_img.shape:
            real_img = cv2.resize(real_img, (gen_img.shape[1], gen_img.shape[0]))

        score, _ = ssim(gen_img, real_img, full=True)
        if score > max_ssim:
            max_ssim = score
            best_real_img_path = real_img_path
    
    highest_ssim_values.append((gen_img_path, max_ssim, best_real_img_path))

# Sort the highest SSIM values in descending order
highest_ssim_values.sort(key=lambda x: x[1], reverse=True)

# Calculate the average SSIM value
total_ssim = sum(value for _, value, _ in highest_ssim_values)
average_ssim = total_ssim / len(highest_ssim_values)

for gen_img_path, value, real_img_path in highest_ssim_values:
    print(f"Generated image {gen_img_path} - Highest SSIM: {value} - Real image: {real_img_path}")

print(f"Average SSIM value: {average_ssim}")



In [ ]:
# Select generated images that have SSIM value > 0.5 
# (can store these images to another folder and apply PSNR metric to select imgs have both SSIM > 0,5 and PSNR > 30)

for value in highest_ssim_values:
    if value[1] > 0.5:
        print(value)

In [ ]:
# PSNR metric

import os
import cv2
import numpy as np

real_images_folder = 'C:/brain_tumor/version2/notebook/classify_dataset/meningioma'
generated_images_folder = 'C:/brain_tumor/version2/notebook/dc_gan_final/meni/meni_img'

def load_images_from_folder(folder):
    images = []
    image_paths = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            images.append(img)
            image_paths.append(img_path)
    return images, image_paths

def compute_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    PIXEL_MAX = 255.0
    return 20 * np.log10(PIXEL_MAX / np.sqrt(mse))

real_images, real_image_paths = load_images_from_folder(real_images_folder)
generated_images, generated_image_paths = load_images_from_folder(generated_images_folder)

highest_psnr_values = []

for gen_img, gen_img_path in zip(generated_images, generated_image_paths):
    max_psnr = -1
    best_real_img_path = None
    for real_img, real_img_path in zip(real_images, real_image_paths):
        if gen_img.shape != real_img.shape:
            real_img = cv2.resize(real_img, (gen_img.shape[1], gen_img.shape[0]))

        score = compute_psnr(gen_img, real_img)
        if score > max_psnr:
            max_psnr = score
            best_real_img_path = real_img_path
    
    highest_psnr_values.append((gen_img_path, max_psnr, best_real_img_path))

# Sort the highest PSNR values in descending order
highest_psnr_values.sort(key=lambda x: x[1], reverse=True)

# Calculate the average PSNR value
total_psnr = sum(value for _, value, _ in highest_psnr_values)
average_psnr = total_psnr / len(highest_psnr_values)

for gen_img_path, value, real_img_path in highest_psnr_values:
    print(f"Generated image {gen_img_path} - Highest PSNR: {value} - Real image: {real_img_path}")

print(f"Average PSNR value: {average_psnr}")



In [ ]:
# Select generated images that have PSNR value > 30 dB

for value in highest_psnr_values:
    if value[1] > 30:
        print(value)